In [1]:
#Importing all the libraries used in the notebooks
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

### Versione iniziale di frag_face

In [2]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [3]:
V = Float64[
    0 0 0; 0 1 0;
    1 1 0; 1 0 0;
    0 0 1; 0 1 1;
    1 1 1; 1 0 1
]

8×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 1.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  1.0
 1.0  1.0  1.0
 1.0  0.0  1.0

In [4]:
EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

12×8 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
 -1   1   ⋅   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅  -1   1   ⋅   ⋅   ⋅   ⋅  ⋅
  ⋅   ⋅  -1   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  ⋅
 -1   ⋅   ⋅   ⋅   1   ⋅   ⋅  ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1   ⋅  ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  ⋅
  ⋅   ⋅   ⋅  -1   ⋅   ⋅   ⋅  1
  ⋅   ⋅   ⋅   ⋅  -1   1   ⋅  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅  -1   1  ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  1
  ⋅   ⋅   ⋅   ⋅  -1   ⋅   ⋅  1

In [5]:
FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

`@btime` riporta i risultati come `@time`, mostrando le allocazioni ed il tempo minimo trascorso in una singola chiamata di funzione durante l'esecuzione della funzione su più campionamenti. Useremo `@btime` nella maggior parte dei nostri esempi.

In [6]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  350.100 μs (4870 allocations: 265.34 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

Utilizzando la macro `@code_warntype` si individuano molte variabili assegnate al tipo `Any`. Questo significa essenzialmente che ci sarà un'allocazione per la posizione della memoria e l'indirezione al valore effettivo durante l'esecuzione della funzione. 

In [7]:
@code_warntype frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face(V, EV, FE, sp_idx, sigma) in Main at In[2]:1
Arguments
  #self#::Core.Const(frag_face)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  @_8::Union{Nothing, Tuple{Int64, Nothing}}
  nvsize::Any
  nFE::Any
  nEV::Any
  nV::Any
  tV::Any
  M::Any
  sEV::Any
  sV::Any
  sigmavs::Vector{Int64}
  vs_num::Int64
  @_19::Any
  @_20::Any
  i::Int64
  tmpEV::Any
  tmpV::Any
Body::Tuple{Any, Any, Any}
1 ─       Core.NewvarNode(:(@_7))
│         Core.NewvarNode(:(nvsize))
│         Core.NewvarNode(:(nFE))
│         Core.NewvarNode(:(nEV))
│         Core.NewvarNode(:(nV))
│         (vs_num = Main.size(V, 1))
│   %7  = (sigma:sigma)::UnitRange{Int64}
│   %8  = Base.getindex(FE, %7, Main.:(:))::SparseMatrixCSC{Int8, Int64}
│   %9  = Base.broadcasted(

Utilizzando la macro `@profile` si immagazzinano le informazioni relative alla funzione che successivamente vengono mostrate graficamente. Tramite il grafico riusciamo a dare una misurazione temporale di ogni singola funzione. La larghezza delle barre mostra il tempo trascorso in ogni locazione di chiamata, mentre la gerarchia di chiamata è rappresentata dalle varie altezze del grafico.

In [8]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    @profile frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [9]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
  ╎66 @Base\task.jl:423; (::IJulia.var"#15#18")()
  ╎ 66 @IJulia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
  ╎  66 @Base\essentials.jl:714; invokelatest
  ╎   66 @Base\essentials.jl:716; #invokelatest#2
  ╎    66 ...c\execute_request.jl:67; execute_request(socket::ZMQ.Soc...
  ╎     66 ...\SoftGlobalScope.jl:65; softscope_include_string(m::Mod...
  ╎    ╎ 66 @Base\loading.jl:1196; include_string(mapexpr::typeo...
  ╎    ╎  66 @Base\boot.jl:373; eval
  ╎    ╎   66 ...ile\src\Profile.jl:28; top-level scope
  ╎    ╎    66 In[8]:4; macro expansion
 1╎    ╎     64 ...le\src\Profile.jl:28; macro expansion
  ╎    ╎    ╎ 7  In[2]:7; frag_face(V::Matrix{Float64...
  ╎    ╎    ╎  1  @Base\broadcast.jl:860; materialize
  ╎    ╎    ╎   1  ...igherorderfns.jl:175; copy
  ╎    ╎    ╎    1  ...gherorderfns.jl:170; _noshapecheck_map(::typeo...
  ╎    ╎    ╎     1  ...gherorderfns.jl:233; _allocres
  ╎    ╎    ╎    ╎ 1  ...parsematrix.jl

# Ottimizzazione



Possiamo assegnare alle variabili locali un tipo deterministico per rimuovere il tipo `Any`.
Inoltre si possono creare delle viste degli array, con la macro `@views`, che permettono di accedere ai valori dell'array senza dover effettuare una copia.

Si nota come utilizzando le macro `@views` e `@async` la funzione frag_face ottiene un nuovo tempo minimo con un decremento di circa 20% rispetto alla funzione originale.

In [10]:
function frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    @views sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    @views nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

frag_face2 (generic function with 1 method)

#### Tempi minimi ottenuti da `@btime` con le diverse macro:

`@async` = 292.500 μs (3873 allocations: 218.65 KiB)

`Threads.@spawn` = 305.200 μs (4093 allocations: 230.38 KiB)

`@simd` = 383.000 μs (4996 allocations: 269.99 KiB)

`@inbounds` = 377.700 μs (4996 allocations: 269.99 KiB)

In [11]:
@btime frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

  283.900 μs (3873 allocations: 218.65 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 1, 2, 2, 3, 3, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, 1, -1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, 1, 1, -1], 1, 4))

In [12]:
@code_warntype frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

MethodInstance for frag_face2(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face2(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, sp_idx::Vector{Int64}, sigma::Int64) in Main at In[10]:1
Arguments
  #self#::Core.Const(frag_face2)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Any
  nvsize::Int64
  nFE::SparseMatrixCSC{Int8, Int64}
  nEV::SparseMatrixCSC{Int8, Int64}
  nV::Matrix{Float64}
  tV::Matrix{Float64}
  M::Matrix{Float64}
  sEV@_14::Core.Box
  sV@_15::Core.Box
  sigmavs::Vector{Int64}
  vs_num::Int64
  #1::var"#1#2"{SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Vector{Int64}, Int64, Matrix{Float64}}
  task::Task
  sV@_20::Union{}
  sV@_21::Union{}
  sV@_22::Union{}
  sEV@_23::Union{}
Body::Tuple{Matrix{Float64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCS

In [13]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [14]:
ProfileView.view()
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
  ╎93 @Base\task.jl:423; (::IJulia.var"#15#18")()
  ╎ 93 @IJulia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
  ╎  93 @Base\essentials.jl:714; invokelatest
  ╎   93 @Base\essentials.jl:716; #invokelatest#2
  ╎    93 ...c\execute_request.jl:67; execute_request(socket::ZMQ.Soc...
  ╎     93 ...\SoftGlobalScope.jl:65; softscope_include_string(m::Mod...
  ╎    ╎ 93 @Base\loading.jl:1196; include_string(mapexpr::typeo...
  ╎    ╎  93 @Base\boot.jl:373; eval
  ╎    ╎   93 ...ile\src\Profile.jl:28; top-level scope
  ╎    ╎    93 In[13]:4; macro expansion
  ╎    ╎     92 In[10]:23; frag_face2(V::Matrix{Float64...
  ╎    ╎    ╎ 92 ...ar_arrangement.jl:583; planar_arrangement(V::Matri...
  ╎    ╎    ╎  65 ...r_arrangement.jl:583; planar_arrangement(V::Matr...
  ╎    ╎    ╎   1  ...r_arrangement.jl:476; planar_arrangement_1(V::M...
  ╎    ╎    ╎    1  ...sparsematrix.jl:1700; spzeros
  ╎    ╎    ╎     1  ...sparsematrix.jl:1703; spzer

In [15]:
@benchmark frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 7931 samples with 1 evaluation.
 Range (min … max):  352.800 μs … 60.619 ms  ┊ GC (min … max):  0.00% … 0.00%
 Time  (median):     507.000 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   625.867 μs ±  1.435 ms  ┊ GC (mean ± σ):  14.43% ± 7.16%

   ▄█▁       ▄▆▄▁                                               
  ▇███▇▆▅▅▅▄▄████▆▅▅▄▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  353 μs          Histogram: frequency by time         1.09 ms <

 Memory estimate: 265.71 KiB, allocs estimate: 4874.

In [16]:
@benchmark frag_face2(Lar.Points(V),EV,FE,[2,3,4,5],2)

BenchmarkTools.Trial: 9007 samples with 1 evaluation.
 Range (min … max):  279.400 μs … 36.982 ms  ┊ GC (min … max):  0.00% … 96.52%
 Time  (median):     363.400 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   551.456 μs ±  1.864 ms  ┊ GC (mean ± σ):  22.33% ±  6.63%

   ▂█▄▁                                                         
  ▃████▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  279 μs          Histogram: frequency by time          936 μs <

 Memory estimate: 218.65 KiB, allocs estimate: 3873.